In [21]:
import os
import openai
from langchain import OpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader

In [22]:
# Load OpenAI API key from environment variable
openai.api_key = os.getenv("OPENAI_API_KEY")

In [23]:
# Step 1: Import the documents from the database
documents_directory = '/Users/vedantjoshi/Desktop/cq_Bot/ContractQuery/Synthetic.Data/'

In [24]:
# Load all text files
text_loader = DirectoryLoader(documents_directory, glob="*.txt", loader_cls=TextLoader)
text_documents = text_loader.load()

# Load all PDF files
pdf_loader = DirectoryLoader(documents_directory, glob="*.pdf", loader_cls=PyPDFLoader)
pdf_documents = pdf_loader.load()

# Combine all documents
documents = text_documents + pdf_documents

In [25]:
# Step 2: Initialize the OpenAI embedding model
embeddings = OpenAIEmbeddings()

In [26]:
# Step 3: Initialize ChromaDB and add documents
vectorstore = Chroma(persist_directory="chromadb_data", embedding_function=embeddings)
vectorstore.add_texts(texts=[doc.page_content for doc in documents], metadatas=[doc.metadata for doc in documents])

# Persist the ChromaDB instance (this step is necessary if you want to use the database in future sessions)
vectorstore.persist()

/Users/vedantjoshi/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [27]:
# Step 4: Create the RetrievalQA chain using Langchain
retrieval_qa = RetrievalQA.from_chain_type(
    llm=OpenAI(temperature=0),  # OpenAI model instance
    chain_type="stuff",  # RAG retrieval strategy
    retriever=vectorstore.as_retriever()
)

/Users/vedantjoshi/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [30]:
# Step 5: Sample Queries for Testing
queries = [
    "What is the weather today?",
]

In [31]:
# Test the RAG system with the sample queries
for query in queries:
    result = retrieval_qa.run(query)
    print(f"Query: {query}")
    print(f"Answer: {result}")
    print("-" * 50)

Query: What is the weather today?
Answer:  I don't know, as the context provided does not mention anything about the weather.
--------------------------------------------------
